# Initial Balance: Study

## By: Royce Anton Jose
>

# 1. Introduction:
>


The Initial Balance is essentially the price range that has been formed during the opening of a trading session. Typically, traders accept it as the high and low of the first hour of trading, however, it's not always the case. The general purpose of this indicator is to form a bias for the session by knowing what the intent of the market is early on.

**Initial Balance - Core Concepts:**
Initial Balance (IB) core concepts revolve around mainly 3 things which help use gain an edge of the market:

- IB Volume
- IB range
- Position of the Daily(1D) Open

**IB Volume:**

If we notice a higher volume within the IB, this indicates bigger traders are active and shows interest for price at that level. This in turn indicates the session would be a range bound session.

<table><tr><td><img src='https://i.gyazo.com/4f1322ffae4789440d118a9d6a53ca5f.png'></td><td><img src='https://i.gyazo.com/a37c2fc9b6c8210f53ac45430e5acc8a.png'></td></tr></table>

On the other hand, a low volume IB indicates that there is less interest in transacting at that level. This increases the likelihood for a trending session.

**IB Range:**

A larger range ties in with larger volume. Similar to high IB volume, this indicates traders are active and interested; leading to a higher chance of a range bound session.

A lower range on the other hand indicates the less interest, which increases the likelihood for a trending session to happen.

**Position of the 1D Open:**

- **Near IB Low:** If we notice the Daily Open near the IB Low and witnesses a price driven all the way up to the IB high, this indicates the market participants drove the price up on open. This 
leads to a higher probability of an uptrending session.


- **Middle of IB:** The price opening at the middle of the IB indicates acceptance of opening price. This increases the chances of a range bound session. We can notice the price action moves on either side getting faded back to the daily open range.


- **Near IB High:** If we notice the Daily Open near the IB High and witnesses a price driven all the way down to the IB low, this indicates the market participants drove the price down on open. This leads to a higher probability of an downtrending session.



# 2. Data Analysis:
>

## 2.1 Import and Clean Data:
>
To study the IB; data for the last `100` days of `BTC-USD` has been imported using the Binance client. Furthermore, the data is then turned into a dataframe using `pandas` library and calculations has been performed using the `numpy` library.

In [232]:
!pip install -U kaleido

In [229]:
!pip install jovian yfinance plotly python-binance -U kaleido --upgrade --quiet

In [225]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import jovian
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import datetime
import requests
from IPython.display import display, Math, Markdown, Image

In [3]:
log = pd.read_csv('binance-key.csv')

apiKey = log['key'][0]
secret = log['key'][1]

client = Client(apiKey, secret)
tickers = client.get_all_tickers()
tickers_df = pd.DataFrame(tickers)
symbols = list(tickers_df['symbol']) #extracting complete symbols for later(optional)

In [69]:
symbol = 'BTCUSDT'

start = datetime.date.today() - datetime.timedelta(days=28)
start = start.strftime("%d %B, %Y")

candlestick_chart = client.get_historical_klines(symbol,
                                                Client.KLINE_INTERVAL_15MINUTE,
                                                start)


#data for candlestick 15min
candlestick_df = pd.DataFrame(candlestick_chart)
candlestick_df.columns = ['Open Time', 'open', 'high', 'low', 'close', 'volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

candlestick_df['Open Time'] = pd.to_datetime(candlestick_df['Open Time'], unit='ms')
candlestick_df['Close Time'] = pd.to_datetime(candlestick_df['Close Time'], unit='ms')

candlestick_df.insert(1,'Close Time', candlestick_df.pop('Close Time')) # move column to 2nd position
candlestick_df['date'] = candlestick_df['Open Time']

candlestick_df.drop(candlestick_df.iloc[:,0:2],inplace=True,axis=1)
candlestick_df.drop(candlestick_df.iloc[:,5:10],inplace=True,axis=1)

candlestick_df = candlestick_df.set_index('date')
candlestick_df = candlestick_df.apply(lambda col:pd.to_numeric(col, errors='coerce'))

candlestick_df['symbol'] = symbol


ohlc_data = candlestick_df


In [5]:
ohlc_data.tail()

,open,high,low,close,volume,symbol
date,,,,,,
2022-05-23 09:30:00,30366.15,30478.87,30366.15,30400.58,493.93797,BTCUSDT
2022-05-23 09:45:00,30400.57,30520.00,30378.82,30441.94,540.82051,BTCUSDT
2022-05-23 10:00:00,30441.94,30556.28,30425.19,30529.99,439.26663,BTCUSDT
2022-05-23 10:15:00,30529.99,30540.46,30426.34,30475.45,443.44064,BTCUSDT
2022-05-23 10:30:00,30475.45,30495.00,30431.34,30434.43,27.98916,BTCUSDT


## 2.2 Creating Initial Balance(IB) Dataframe:

We now have extracted a dataset named `ohlc_data` for `BTC-USDT` with the required set of columns on which the IB calculations needs to be performed. From the extracted dataset, we will create a new dataset `initial_balance` which extracts the first hour `high` and `low` of each day which is used to calculate the initial balance high `ib_h`, low `ib_l`, 1st hour volume `ib_vol` and initial balnce width 'ib_width'; a numberic difference value between hourly high and low on the original dataframe
>

In [115]:
initial_balance = (ohlc_data[ohlc_data.index.time < datetime.time(1,00)].resample('D')
                   .apply({'high':'max','low':'min','volume': 'sum'}).dropna())


initial_balance['symbol'] = symbol

column_to_move = initial_balance.pop('symbol')
initial_balance.insert(0, 'symbol', column_to_move)


In [116]:
initial_balance['width'] = initial_balance['high'].astype(float) - initial_balance['low'].astype(float)
initial_balance['mid'] =  (initial_balance['high'].astype(float) + initial_balance['low'].astype(float))/2

In [117]:
ohlc_data['ib_h'] = ohlc_data.index.map(
    lambda x: initial_balance.loc[pd.to_datetime(x.date())]['high'] if x.time()>=datetime.time(1,00) else np.nan)

ohlc_data['ib_l'] = ohlc_data.index.map(
    lambda x: initial_balance.loc[pd.to_datetime(x.date())]['low']  if x.time()>=datetime.time(1,00) else np.nan)

ohlc_data['ib_vol'] = ohlc_data.index.map(
    lambda x: initial_balance.loc[pd.to_datetime(x.date())]['volume']  if x.time()>=datetime.time(1,00) else np.nan)

ohlc_data['ib_width'] = ohlc_data.index.map(
    lambda x: initial_balance.loc[pd.to_datetime(x.date())]['width']  if x.time()>=datetime.time(1,00) else np.nan)

In [118]:
initial_balance.tail()

,symbol,high,low,volume,width,mid
date,,,,,,
2022-05-19,BTCUSDT,29056.56,28700.01,2025.92751,356.55,28878.285
2022-05-20,BTCUSDT,30363.69,30142.93,1324.28374,220.76,30253.310
2022-05-21,BTCUSDT,29347.52,29164.95,803.22404,182.57,29256.235
2022-05-22,BTCUSDT,29505.00,29396.20,528.17425,108.80,29450.600
2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545


>
From the calculated values, the first four rows of each dataset are removed and filled `NaN`values as the IB only reflects after the first hour of the day. Our original dataset comprises of `15m` data, essentially grouping the first 4 rows of data to form the hourly OHLC data.
>

## 2.3 Setting up IB Width:
>

To study the IB width, we will apply the same logic behind Bollinger Bands calculations. We will consider a period of past 21 days candle and calculate the average width of the past 21 days. We will also calculate Standard Deviations `ib_sd` of the past 21 day period. 

With this data we can classify the width of the initial balance as follows:

In [10]:
window =21 
ib_avg_width = initial_balance['width'].rolling(window=window).mean()
ib_sd = initial_balance['width'].rolling(window=window).std()

initial_balance['narrow_ib'] =initial_balance['width'] < ib_avg_width - ib_sd
initial_balance['moderate_ib'] = (initial_balance['width']>= ib_avg_width-ib_sd) & (initial_balance['width']<= ib_avg_width+ib_sd)
initial_balance['wide_ib'] =initial_balance['width'] > ib_avg_width + ib_sd


In [11]:
initial_balance.tail()

,symbol,high,low,volume,width,mid,narrow_ib,moderate_ib,wide_ib
date,,,,,,,,,
2022-05-19,BTCUSDT,29056.56,28700.01,2025.92751,356.55,28878.285,False,True,False
2022-05-20,BTCUSDT,30363.69,30142.93,1324.28374,220.76,30253.310,False,True,False
2022-05-21,BTCUSDT,29347.52,29164.95,803.22404,182.57,29256.235,False,True,False
2022-05-22,BTCUSDT,29505.00,29396.20,528.17425,108.80,29450.600,False,True,False
2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,False,True,False


## 2.4 Setting up IB Range:
>

To study the IB Range, consider the daily high and low to the initial balance high and lows. This helps us determine the days where the price remained bound to the range of the Initial Balance.

In [12]:
daily_ohlc_data = ohlc_data.resample('D').apply(
    {
        'open':'first',
        'high':'max',
        'low' : 'min',
        'close': 'last',
        'volume': 'sum'
    }
).dropna()

In [13]:
range_bound = (daily_ohlc_data['high']== initial_balance['high']) & (daily_ohlc_data['low'] == initial_balance['low'])

In [14]:
buffer = 0.25

range_bound = (daily_ohlc_data['high'] <= initial_balance['high'] + ib_avg_width * buffer
              )&(daily_ohlc_data['low'] >= initial_balance['low'] - ib_avg_width * buffer)

### Check IB range Broken Status:

In [15]:
initial_balance['ib_range_break'] = (daily_ohlc_data['high'] > initial_balance['high']) | (daily_ohlc_data['low'] < initial_balance['low'])
initial_balance['ib_range_break'] = initial_balance.apply(lambda x: 'broken' if x.ib_range_break == True else 'inside', axis=1)

In [16]:
initial_balance.tail()

,symbol,high,low,volume,width,mid,narrow_ib,moderate_ib,wide_ib,ib_range_break
date,,,,,,,,,,
2022-05-19,BTCUSDT,29056.56,28700.01,2025.92751,356.55,28878.285,False,True,False,broken
2022-05-20,BTCUSDT,30363.69,30142.93,1324.28374,220.76,30253.310,False,True,False,broken
2022-05-21,BTCUSDT,29347.52,29164.95,803.22404,182.57,29256.235,False,True,False,broken
2022-05-22,BTCUSDT,29505.00,29396.20,528.17425,108.80,29450.600,False,True,False,broken
2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,False,True,False,broken


### Check latest IB and Price Status:

In [17]:
initial_balance['above_IB'] = ohlc_data['high'].tail(1).values > initial_balance['high'].tail(1)
initial_balance['below_IB'] = ohlc_data['low'].tail(1).values < initial_balance['low'].tail(1)
initial_balance['inside_IB'] = (ohlc_data['high'].tail(1).values < initial_balance['high'].tail(1)) & (ohlc_data['low'].tail(1).values > initial_balance['low'].tail(1))

initial_balance['current_status'] = initial_balance.apply( lambda x: 'above IB' if x.above_IB == True else ( 'below IB' if x.below_IB == True else 'inside'), axis =1)

initial_balance = initial_balance.drop(['above_IB','below_IB','inside_IB'], axis =1)

In [18]:
initial_balance.tail(1)

,symbol,high,low,volume,width,mid,narrow_ib,moderate_ib,wide_ib,ib_range_break,current_status
date,,,,,,,,,,,
2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,False,True,False,broken,above IB


### Comparing IB Range to the IB Width:

How many time price traded within IB range on **wide** Initial Balance day:

In [19]:
(initial_balance['wide_ib'] & range_bound).sum()

0

How many times IB was broken on **narrow** Initial Balance day:

In [20]:
(initial_balance['narrow_ib']&~range_bound).sum()

0

How many time IB was broken on **moderate** Initial Balance day:

In [21]:
(initial_balance['moderate_ib']&~range_bound).sum()

9

## 2.5 IB Range Open:

Setting the conditions where the initial balance open is less than initial balance mid as a lower open: 

In [22]:
initial_balance['range_open'] = daily_ohlc_data['close'].shift(+1) < initial_balance['mid']
initial_balance['range_open'] = initial_balance['range_open'].apply(lambda x: 'Bottom' if x == True else 'Top')

## 2.6 IB Width Summary:

A dataframe column named `width_status` that lists the overall width status of ticker on the given day

In [23]:
initial_balance['width_status'] = initial_balance.apply(lambda x: 'narrow' if x.narrow_ib == True else ( 'moderate' if x.moderate_ib == True else 'wide'), axis =1)

In [24]:
initial_balance = initial_balance.drop(['narrow_ib','moderate_ib','wide_ib'], axis=1)

In [25]:
initial_balance[20:]

,symbol,high,low,volume,width,mid,ib_range_break,current_status,range_open,width_status
date,,,,,,,,,,
2022-05-15,BTCUSDT,30273.21,29933.73,1993.34578,339.48,30103.470,broken,inside,Bottom,moderate
2022-05-16,BTCUSDT,31328.90,31048.00,2286.18686,280.90,31188.450,broken,inside,Top,moderate
2022-05-17,BTCUSDT,30227.79,29807.16,1790.10534,420.63,30017.475,broken,inside,Bottom,moderate
2022-05-18,BTCUSDT,30709.99,30416.34,1512.07660,293.65,30563.165,broken,inside,Bottom,moderate
2022-05-19,BTCUSDT,29056.56,28700.01,2025.92751,356.55,28878.285,broken,inside,Bottom,moderate
2022-05-20,BTCUSDT,30363.69,30142.93,1324.28374,220.76,30253.310,broken,inside,Top,moderate
2022-05-21,BTCUSDT,29347.52,29164.95,803.22404,182.57,29256.235,broken,inside,Bottom,moderate
2022-05-22,BTCUSDT,29505.00,29396.20,528.17425,108.80,29450.600,broken,inside,Bottom,moderate
2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,broken,above IB,Bottom,moderate


## 2.7 Setting up IB Volume:
>


### IB Volume Status:

A dataframe column named `volume_status` that lists the overall volume status of ticker on the given day. The volume of the first 1 hour is compared to the previous day's last 1 hour df `vol_balance`. Once compared, the original dataframe is shifted by one row to align with time series and then shifted back to the original table:

## 2.8 Opportunities:
>
Based on the 1 Hr Open and IB range open, a general idea for the opportunities to look for the day trading can be derived. If the range Open is high, it generally indicates the market price got pushed down in the first hour, indicating stronger chances for downtrend. Hence we prefer to take short opportunities than long for that specific day. 

In [26]:
initial_balance['opportunity'] = initial_balance.apply(lambda x: 'short' if x.range_open == 'Top' else 'long', axis=1)

In [27]:
#extracting last 1 hr volume data
vol_balance = (ohlc_data[ohlc_data.index.time > datetime.time(23,00)].resample('D')
                   .apply({'high':'max','low':'min','volume': 'sum'}))


vol_balance['symbol'] = symbol

column_to_move = vol_balance.pop('symbol')
vol_balance.insert(0, 'symbol', column_to_move)


In [28]:
initial_balance['volume_status'] = vol_balance['volume'] < initial_balance['volume'].shift(-1)[:-1]
initial_balance['volume_status'] = initial_balance['volume_status'].shift(1).apply(lambda x: 'high' if x== True else 'low')


In [29]:
initial_balance

,symbol,high,low,volume,width,mid,ib_range_break,current_status,range_open,width_status,opportunity,volume_status
date,,,,,,,,,,,,
2022-04-25,BTCUSDT,39500.84,38674.00,4228.78013,826.84,39087.420,broken,inside,Top,wide,short,low
2022-04-26,BTCUSDT,40773.65,40387.55,2178.73925,386.10,40580.600,broken,inside,Bottom,wide,long,high
2022-04-27,BTCUSDT,38261.17,37881.31,2328.07957,379.86,38071.240,broken,inside,Top,wide,short,low
2022-04-28,BTCUSDT,39455.00,39152.96,1321.56279,302.04,39303.980,broken,inside,Bottom,wide,long,high
2022-04-29,BTCUSDT,39869.28,39675.46,1133.07982,193.82,39772.370,broken,inside,Bottom,wide,long,high
2022-04-30,BTCUSDT,38795.38,38551.12,1150.93059,244.26,38673.250,broken,inside,Bottom,wide,long,high
2022-05-01,BTCUSDT,37806.79,37500.00,2313.83572,306.79,37653.395,broken,inside,Bottom,wide,long,low
2022-05-02,BTCUSDT,38810.73,38448.84,2460.83144,361.89,38629.785,broken,inside,Bottom,wide,long,high
2022-05-03,BTCUSDT,38573.82,38384.60,1405.58114,189.22,38479.210,broken,inside,Top,wide,short,high


# 3. Initial Balance: Interactive Visualization:
>

### Manipulating Dataframe to extract desired visualization:
>
First we set up the initail dataframe to show the OHLC candles for the recent dates by locating a recent segment of the dataframe:

In [208]:
plot_df

,date,open,high,low,close,volume,symbol,ib_h,ib_l,ib_vol,ib_width
0,2022-05-21 01:00:00,29279.16,29300.00,29182.76,29230.63,200.01481,BTCUSDT,29347.52,29164.95,803.22404,182.57
1,2022-05-21 01:15:00,29230.63,29249.55,28947.28,28969.17,711.25364,BTCUSDT,29347.52,29164.95,803.22404,182.57
2,2022-05-21 01:30:00,28969.17,29138.10,28969.17,29113.98,347.19035,BTCUSDT,29347.52,29164.95,803.22404,182.57
3,2022-05-21 01:45:00,29113.98,29188.68,29070.00,29150.00,232.51204,BTCUSDT,29347.52,29164.95,803.22404,182.57
4,2022-05-21 02:00:00,29150.00,29179.40,29076.14,29101.86,162.10565,BTCUSDT,29347.52,29164.95,803.22404,182.57
...,...,...,...,...,...,...,...,...,...,...,...
230,2022-05-23 10:30:00,30475.45,30495.00,30419.43,30472.79,230.35921,BTCUSDT,30459.56,30193.53,1515.75247,266.03
231,2022-05-23 10:45:00,30472.80,30510.00,30410.20,30484.01,400.45292,BTCUSDT,30459.56,30193.53,1515.75247,266.03
232,2022-05-23 11:00:00,30484.00,30616.18,30466.85,30583.55,605.39496,BTCUSDT,30459.56,30193.53,1515.75247,266.03
233,2022-05-23 11:15:00,30583.55,30588.29,30501.65,30532.32,414.47168,BTCUSDT,30459.56,30193.53,1515.75247,266.03


In [235]:
plot_df = candlestick_df.iloc[2580:,:].reset_index()

fig = go.Figure(data = [go.Candlestick(x= plot_df['date'],
                                      open = plot_df['open'],
                                      high = plot_df['high'],
                                      low = plot_df['low'],
                                      close = plot_df['close'],
                                      name = 'Price Candle',
                                      increasing_line_color='#FFFFFF', 
                                      decreasing_line_color='#000000',
                                      legendgroup= 'one')]) # use #2196F3 for dark theme
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(plot_bgcolor='#FFFFFF') # use #131722 for dark theme
fig.update_xaxes(showgrid= False)
fig.update_yaxes(showgrid= False)


fig.update_traces(increasing_line_width=1, 
                  selector=dict(type='candlestick'))
fig.update_traces(increasing_fillcolor='#FFFFFF', 
                  selector=dict(type='candlestick'))
fig.update_traces(increasing_line_color='#000000', 
                  selector=dict(type='candlestick'))


fig.update_traces(decreasing_line_width=1, 
                  selector=dict(type='candlestick'))
fig.update_traces(decreasing_fillcolor='#000000', 
                  selector=dict(type='candlestick'))

fig.update_traces(showlegend= False, selector=dict(type='scatter'))

fig.add_trace(go.Scatter(mode='lines',
                             x= plot_df['date'],
                             y= plot_df['ib_h'],
                             line=dict(color='purple', width=1),
                             name='high',
                             showlegend=True,))

fig.add_trace(go.Scatter(mode='lines',
                             x= plot_df['date'],
                             y= plot_df['ib_l'],
                             line=dict(color='purple', width=1),
                             name='low',
                             showlegend=True))

fig.update_layout(autosize=False,
                  height=600, width=850,
                  legend=dict(orientation="h",
                              yanchor="bottom", y=1.02,
                              xanchor="right", x=1))


![image.png](https://i.gyazo.com/5a69014f295a6951f0f104f6aaacb762.png)

# 4. DataFrame Results:
>


In [35]:
initial_balance

,date,symbol,high,low,volume,width,mid,ib_range_break,current_status,range_open,width_status,opportunity,volume_status
0,2022-05-22,BTCUSDT,29505.00,29396.20,528.17425,108.80,29450.600,broken,inside,Bottom,moderate,long,high
1,2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,broken,above IB,Bottom,moderate,long,high


### Latest day IB Status:

In [36]:
initial_balance.tail(1)

,date,symbol,high,low,volume,width,mid,ib_range_break,current_status,range_open,width_status,opportunity,volume_status
1,2022-05-23,BTCUSDT,30459.56,30193.53,1515.75247,266.03,30326.545,broken,above IB,Bottom,moderate,long,high


# 5. References & Resources:

>

- [Binance Futures Python Repo](https://github.com/Binance-docs/Binance_Futures_python)
- [StackOverflow](https://stackoverflow.com/)
- [StreamAlpha](https://www.youtube.com/c/StreamAlpha)
- [Python Binance API](https://python-binance.readthedocs.io/en/latest/binance.html)
- [Binance Docs](https://binance-docs.github.io/apidocs/futures/en/#24hr-ticker-price-change-statistics)


Python Libraries I commonly use:
- [Binance Python](https://python-binance.readthedocs.io/en/latest/)
- [Pandas](https://pandas.pydata.org/docs/) 
- [NumPy](https://numpy.org/doc/stable/)
- [StackOverflow](https://stackoverflow.com/)
- [GeeksForGeeks](https://www.geeksforgeeks.org/)



# 6. Contact me:
>
Feel free to reach out to me. I am currently looking to volunteer for Exploratory Data Analysis and python projects to improve my skill. If you have an questions about the notebook, reach out to on me on my LinkedIn or github:

- [LinkedIn](https://www.linkedin.com/in/royceantonjose/)
- [Github](https://github.com/ranton95)